<a href="https://colab.research.google.com/github/tliones/blank-app/blob/main/Generation_Ratev3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from math import sqrt

In [2]:
#Generate Random Values for Inputs (v, Temp, Length, Width)
num_samples = 10000
MW = 92.1
Patm = 101325
Pv = 2800
Vkin = 1.56 * 10**-5
R = 8.3144

def Temp_Conversion(F):
  return (F - 32) * 5/9 + 273.15


def Chemical_Generation_Rate(min_Vw, max_Vw, min_Ta, max_Ta, min_As, max_As):

  #Generate Random Variables from Inputs
  v_wind_samples = np.random.uniform(min_Vw, max_Vw, num_samples)
  Air_Temp_samples = np.random.uniform(Temp_Conversion(min_Ta), Temp_Conversion(max_Ta), num_samples)
  As_samples = np.random.uniform(min_As, max_As, num_samples)

  #Compute Molecular Diffusion Coefficient (m2/s)

  Diff_C = (4.14 * 10**-4 * Air_Temp_samples**1.9 * np.sqrt((1/29)+(1/MW)) * MW**-0.33) / Patm

  G_Rate = 1300 * Diff_C**2 * Vkin**-.9 * ((100 * Vkin * v_wind_samples) / Diff_C)**(0.625*(Vkin / Diff_C)**0.3)*np.sqrt(As_samples)**-0.11 * np.sqrt(Patm / (Patm - Pv)) * Pv / (R * Air_Temp_samples) * MW * As_samples

  return G_Rate


In [3]:
!pip install thermo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 627.5/627.5 kB 31.0 MB/s eta 0:00:00


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from thermo.chemical import Chemical

# Define a dictionary with chemical names as keys
chemicals = {
    'Water': {'name': 'Water'},
    'Ethanol': {'name': 'Ethanol'},
    'Benzene': {'name': 'Benzene'},
    'Acetone': {'name': 'Acetone'}
}

# Generate Random Values for Inputs (v, Temp, Length, Width)
num_samples = 10000
Patm = 101325
Vkin = 1.56 * 10**-5
R = 8.3144

def Temp_Conversion(F):
    return (F - 32) * 5/9 + 273.15

def Chemical_Generation_Rate(min_Vw, max_Vw, min_Ta, max_Ta, min_As, max_As, chem_name):
    chemical = Chemical(chem_name)
    MW = chemical.MW
    v_wind_samples = np.random.uniform(min_Vw, max_Vw, num_samples)
    Air_Temp_samples = np.random.uniform(Temp_Conversion(min_Ta), Temp_Conversion(max_Ta), num_samples)
    vapor_pressure_func = np.vectorize(chemical.VaporPressure)
    Pv_samples = vapor_pressure_func(Air_Temp_samples)
    As_samples = np.random.uniform(min_As, max_As, num_samples)
    Diff_C = (4.14 * 10**-4 * Air_Temp_samples**1.9 * np.sqrt((1/29)+(1/MW)) * MW**-0.33) / Patm
    G_Rate = 1300 * Diff_C**2 * Vkin**-.9 * ((100 * Vkin * v_wind_samples) / Diff_C)**(0.625*(Vkin / Diff_C)**0.3)*np.sqrt(As_samples)**-0.11 * np.sqrt(Patm / (Patm - Pv_samples)) * Pv_samples / (R * Air_Temp_samples) * MW * As_samples
    return G_Rate

# Define interactive widgets for input
chemical_widget = widgets.Dropdown(options={key: val['name'] for key, val in chemicals.items()}, description='Chemical:')
min_Vw_widget = widgets.FloatSlider(value=0, min=0, max=20, step=0.1, description="Min Vw (m/s):")
max_Vw_widget = widgets.FloatSlider(value=20, min=0, max=20, step=0.1, description="Max Vw (m/s):")
min_Ta_widget = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description="Min Ta (F):")
max_Ta_widget = widgets.FloatSlider(value=100, min=0, max=100, step=0.1, description="Max Ta (F):")
min_As_widget = widgets.FloatSlider(value=0, min=0, max=10, step=0.1, description="Min As (m2):")
max_As_widget = widgets.FloatSlider(value=10, min=0, max=20, step=0.1, description="Max As (m2):")

# Function to update and plot
def update_plot(chem_name, min_Vw, max_Vw, min_Ta, max_Ta, min_As, max_As):
    G_Rate = Chemical_Generation_Rate(min_Vw, max_Vw, min_Ta, max_Ta, min_As, max_As, chem_name)
    D_95th_percentile = np.percentile(G_Rate, 95)
    plt.figure(figsize=(10, 5))
    plt.hist(G_Rate, bins=50, alpha=0.7, color='blue', edgecolor='black', density=True, label="Distribution of G")
    plt.axvline(D_95th_percentile, color='red', linestyle='dashed', linewidth=2, label=f'95th Percentile: {D_95th_percentile:.2e}')
    plt.xlabel("Generation Rate (G) (gm/min)")
    plt.ylabel("Density")
    plt.title("Distribution of the Generation Rate (G)")
    plt.legend()
    plt.show()

# Create an interactive UI
interactive_plot = widgets.interactive(update_plot,
                                       chem_name=chemical_widget,
                                       min_Vw=min_Vw_widget, max_Vw=max_Vw_widget,
                                       min_Ta=min_Ta_widget, max_Ta=max_Ta_widget,
                                       min_As=min_As_widget, max_As=max_As_widget)

display(interactive_plot)


interactive(children=(Dropdown(description='Chemical:', options={'Water': 'Water', 'Ethanol': 'Ethanol', 'Benz…